In [2]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/djdelrio/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [1]:
import pandas as pd
from scraper_functions import has_no_words
import re

In [3]:
from nltk.corpus import stopwords
stopwords = stopwords.words('english')
# stopwords.remove('no')
# stopwords.remove('not')
fil_stopwords = ['ang', 'po', 'naman', 'ito', 'medyo', 'may', 'yung', 'kasi', 'sa', 'na', 'ng', 'ko', 'ba', 'ako', 'lang']
for word in fil_stopwords:
    stopwords.append(word)

In [4]:
file_name = 'tika_transcript'

df = pd.read_csv('data/{}.csv'.format(file_name), index_col=0)
df.shape

(420, 3)

In [5]:
df['date'] = pd.to_datetime(df['date'], format='%B %d, %Y')
df = df.sort_values('date', ascending=False).reset_index().drop('index', axis=1)
df.head(10)

,title,date,text
0,Talk to the People of President Rodrigo Roa Du...,2020-05-04,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...
1,Talk to the People of President Rodrigo Roa Du...,2020-04-27,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...
2,Talk to the People of President Rodrigo Roa Du...,2020-04-24,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...
3,Talk to the People of President Rodrigo Roa Du...,2020-04-16,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...
4,Intervention of President Rodrigo Roa Duterte ...,2020-04-14,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...
5,Intervention of President Rodrigo Roa Duterte ...,2020-04-14,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...
6,Talk to the People of President Rodrigo Roa Du...,2020-04-13,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...
7,Meeting of President Rodrigo Roa Duterte with ...,2020-04-08,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...
8,Talk to the Nation of President Rodrigo Roa Du...,2020-04-06,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...
9,Talk to the Nation on COVID-19 of President Ro...,2020-04-03,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...


In [6]:
# Example of speech; remove everything before the date
entry = df.loc[150]
sample = entry['text']
index = sample.find(']')
print(entry['title'])
print(sample[index+1:1000])
print('===============================')
print(sample[-1000:])

President Rodrigo Roa Duterte at “Give Us This Day” by Pastor Apollo C. Quiboloy
 

  

  
PRESIDENT DUTERTE: …flood the country with drugs, it really means the 

end of it. It’s a deadlock. 

  

Mexico, there is the [Sinaloa]. There is the cartel versus Mexico. I would not 

want that to happen. 

  

There’s a very strange side story in this, Pastor. Alunan, Secretary Alunan, 

one of the brightest of the Cabinet --- among the Cabinet members of 

President Ramos. Alunan was called by the DEA in America --- that is the 

Drug Enforcement Agency and he was told point-blank that if your country 

cannot improve on its present situation combatting drugs, it would not be a 

surprise thing for us if your country becomes a narco state. 

  
An
ry own. 
We want him to know that the congregation of the Kingdom is always 

praying for your good health, for your security, for your long life, 

and the continuous success of your crusade that you have promised the 

Filipino people and that ex

## Cleaning the Data

In [11]:
def clean_data(text):
    if not isinstance(text, float):
        index_of_date = text.find(']')
        text = text[index_of_date+1:]
        text = text.lower()
    #     I included the asterisk * to include swears
        text = re.sub(r'([^A-Z a-z*\'\[\]])|', '', text)

    #     Not sure if I will still remove stopwords
        text = ' '.join([word for word in text.split() if word not in stopwords])

        text = text.replace('presidential news desk', '')
        text = text.replace('pcoopnd', '')

        text = ' '.join(text.split())
    
    return text

In [12]:
df['clean_text'] = df['text'].apply(clean_data)

In [13]:
' '.join(df.loc[3]['clean_text'].split()[:100])

'mga kapatid kong pilipino nandito harap ninyo reklamo hindi ngayon kayo akin kung magreklamo inyo una itong quarantine natin enhanced ano really intended protect guy kung ayaw mo maprotektahan ka reason behind kung bakit ka pinipigilan para kapwa tao mo sense kung ayaw talaga ninyong maniwala nagmamatigas ulo ninyo kulang disiplina aba eh talagang hindi mahihinto every time magdidikit kayo eh nakita nagsasabong kayo painom pa know im calling local government units kindly follow direction guidelines hindi nga namimilit hindi nga martial law eh even martial law put people prison walang rason martial law wala namang virus diyan wala namang'

## Saving cleaned text as CSV

In [17]:
# Remove NaN values
df = df.dropna()

In [18]:
df.to_csv('data/{}_clean.csv'.format(file_name))
df.head()

,title,date,text,clean_text
0,Talk to the People of President Rodrigo Roa Du...,2020-05-04,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,president duterte reduced changed way life eve...
1,Talk to the People of President Rodrigo Roa Du...,2020-04-27,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,dost secretary fortunato dela pea susunod kahi...
2,Talk to the People of President Rodrigo Roa Du...,2020-04-24,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,presidential spokesperson harry roque magandan...
3,Talk to the People of President Rodrigo Roa Du...,2020-04-16,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,mga kapatid kong pilipino nandito harap ninyo ...
4,Intervention of President Rodrigo Roa Duterte ...,2020-04-14,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,thank mr chair majesty excellencies welcome le...
